In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import sys

Using TensorFlow backend.


In [2]:
import preprocess as ppx

In [3]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names)
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

[[[211 202 171]
  [213 204 173]
  [217 208 177]
  ..., 
  [225 213 191]
  [222 210 188]
  [220 208 186]]

 [[241 232 201]
  [241 232 201]
  [243 234 203]
  ..., 
  [227 215 193]
  [225 213 191]
  [223 211 189]]

 [[244 235 204]
  [246 237 206]
  [248 239 208]
  ..., 
  [223 211 189]
  [223 211 189]
  [223 211 189]]

 ..., 
 [[249 246 213]
  [237 234 201]
  [230 227 194]
  ..., 
  [247 233 204]
  [247 233 204]
  [246 232 203]]

 [[255 253 220]
  [243 240 207]
  [235 232 199]
  ..., 
  [246 232 203]
  [246 232 203]
  [245 231 202]]

 [[230 227 194]
  [240 237 204]
  [247 244 211]
  ..., 
  [245 231 202]
  [243 229 200]
  [241 227 198]]]
[[[234 236 225]
  [236 238 227]
  [238 238 228]
  ..., 
  [235 232 225]
  [235 232 225]
  [235 232 227]]

 [[235 236 228]
  [236 238 227]
  [237 237 227]
  ..., 
  [233 230 223]
  [237 234 227]
  [237 234 229]]

 [[233 234 226]
  [238 238 230]
  [238 238 230]
  ..., 
  [234 231 224]
  [240 237 230]
  [240 237 232]]

 ..., 
 [[240 238 217]
  [240 238 217]


In [4]:
imgs = np.array(train_images)

In [5]:
labels = np.array([ls[1] for ls in train_data])



encoder = preprocessing.LabelEncoder()
encoder.fit(labels)


encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)

print(labels_onehot)

[251 107 226 ...,  77 269 135]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [6]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [7]:
# epoch szám
nb_epoch=100

In [8]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################



# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót
x = base_model.output
x = GlobalAveragePooling2D()(x)
# ezután hozzáadunk egy előrecsatolt réteget ReLU aktivációs függvénnyel
x = Dense(1024, activation='relu')(x)
# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(x)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

In [9]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
labels_onehot.shape[1]

894

In [13]:
model.fit(imgs, labels_onehot, batch_size=8, nb_epoch=10, validation_split=0.2, callbacks=[history])

Train on 1844 samples, validate on 462 samples
Epoch 1/10
1844/1844 [==============================] - 1812s - loss: 7.1727 - acc: 0.0781 - val_loss: 9.9532 - val_acc: 0.0173
Epoch 2/10
1844/1844 [==============================] - 1395s - loss: 4.1563 - acc: 0.2912 - val_loss: 10.3072 - val_acc: 0.0281
Epoch 3/10
1844/1844 [==============================] - 1077s - loss: 1.7809 - acc: 0.6247 - val_loss: 10.9534 - val_acc: 0.0152
Epoch 4/10
1844/1844 [==============================] - 1079s - loss: 0.9181 - acc: 0.7722 - val_loss: 11.7134 - val_acc: 0.0325
Epoch 5/10
1844/1844 [==============================] - 1088s - loss: 0.7263 - acc: 0.8145 - val_loss: 12.0979 - val_acc: 0.0346
Epoch 6/10
1844/1844 [==============================] - 1092s - loss: 0.6019 - acc: 0.8465 - val_loss: 12.4560 - val_acc: 0.0216
Epoch 7/10
1844/1844 [==============================] - 1176s - loss: 0.5767 - acc: 0.8574 - val_loss: 12.7725 - val_acc: 0.0216
Epoch 8/10
1844/1844 [=============================

In [14]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

In [ ]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(np.asarray(train_images), labels_onehot,  batch_size=16, nb_epoch=10, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

In [36]:
predicted_values = model.predict(np.asarray(train_images))

In [57]:
encoder.inverse_transform(max_pred).shape

(770, 459)

In [51]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
print()

In [ ]:
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])
                .reshape(n_samples, n_features) - ohc.feature_indices_[:-1]

In [50]:
np.zeros((3,4), dtype=int)

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [39]:
xx = np.array([4,56,67.3,7])

In [59]:
predicted_values.shape

(770, 459)